# 시카고 샌드위치 맛집 분석

### 1. 시카고 샌드위치 맛집 사이트에 접근하기

In [41]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [42]:
url_base = 'https://www.chicagomag.com'
url_sub = '/Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/'
url = url_base+url_sub
html = urlopen(url)
soup = BeautifulSoup(html, 'html.parser')

In [43]:
rest_list = soup.select('.sammy')
rest_list[0]

<div class="sammy" style="position: relative;">
<div class="sammyRank">1</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/"><b>BLT</b><br>
Old Oak Tap<br>
<em>Read more</em> </br></br></a></div>
</div>

In [44]:
len(rest_list)

50

### 2. 접근한 웹 페이지에서 원하는 데이터 추출하고 정리하기

In [45]:
rank = rest_list[0].select_one('.sammyRank').get_text()
rank

'1'

In [46]:
# 조인을 할 때에는 겹치는부분은 자동적으로 ㅎ마쳐진다.
from urllib.parse import urljoin
url = urljoin(url_base, rest_list[0].find('a').attrs['href'])
url

'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'

In [47]:
tmp = rest_list[0].find('a').get_text()
tmp

'BLT\r\nOld Oak Tap\nRead more '

In [48]:
tmp.split('\n')

['BLT\r', 'Old Oak Tap', 'Read more ']

In [49]:
menu = tmp.split('\n')[0].replace('\r', '')
cafe = tmp.split('\n')[1]
#menu = rest_list[0].find('b').get_text()
menu, cafe

('BLT', 'Old Oak Tap')

In [50]:
rank_list = [] ; url_list = []; menu_list = []; cafe_list = []

In [51]:
for rest in rest_list:
    rank = int(rest.select_one('.sammyRank').get_text())
    url = urljoin(url_base, rest.find('a').attrs['href'])
    tmp = rest.find('a').get_text().split('\n')
    menu = tmp[0].replace('\r', '')
    cafe = tmp[1]

    rank_list.append(rank)
    url_list.append(url)
    menu_list.append(menu)
    cafe_list.append(cafe)

In [52]:
import pandas as pd
df = pd.DataFrame({
    'Rank':rank_list,
    'Cafe':cafe_list,
    'Menu':menu_list,
    'URL':url_list
})
df.head()

,Rank,Cafe,Menu,URL
0,1,Old Oak Tap,BLT,https://www.chicagomag.com/Chicago-Magazine/No...
1,2,Au Cheval,Fried Bologna,https://www.chicagomag.com/Chicago-Magazine/No...
2,3,Xoco,Woodland Mushroom,https://www.chicagomag.com/Chicago-Magazine/No...
3,4,Al’s Deli,Roast Beef,https://www.chicagomag.com/Chicago-Magazine/No...
4,5,Publican Quality Meats,PB&L,https://www.chicagomag.com/Chicago-Magazine/No...


In [53]:
df.to_csv('Chicago.csv', sep=',', encoding='utf8')

### 3. 다수의 웹 페이지에 자동으로 접근해서 원하는 정보 가져오기

In [54]:
html = urlopen(df['URL'][0])
soup_tmp = BeautifulSoup(html, 'html.parser')

In [65]:
soup_tmp.select_one('.addy')

<p class="addy">
<em>$6.85. 3351 N. Broadway, 773-868-4000, <a href="http://phoebesbakery.com/">phoebesbakery.com</a></em></p>

In [66]:
soup_tmp.select_one('.addy').get_text()

'\n$6.85. 3351 N. Broadway, 773-868-4000, phoebesbakery.com'

In [67]:
tmp = soup_tmp.select_one('.addy').get_text().split(',')
len(tmp)

3

In [64]:
tmp[0]

'\n$6.85. 3351 N. Broadway'

In [75]:
price = tmp[0].split()[0][:-1]
price

'$6.85'

In [74]:
addr = ' '.join(tmp[0].split()[1:])
addr

'3351 N. Broadway'

In [71]:
# tel
tmp[1].strip()

'773-868-4000'

In [59]:
from tqdm import tqdm_notebook

In [83]:
rank_list = [] ; url_list = []; menu_list = []; cafe_list = []
price_list=[]; addr_list=[]; tel_list=[]; hp_list=[]
for rest in tqdm_notebook(rest_list):
    rank = int(rest.select_one('.sammyRank').get_text())
    url = urljoin(url_base, rest.find('a').attrs['href'])
    tmp = rest.find('a').get_text().split('\n')
    menu = tmp[0].replace('\r', '')
    cafe = tmp[1]

    rank_list.append(rank)
    url_list.append(url)
    menu_list.append(menu)
    cafe_list.append(cafe)

    html = urlopen(url)
    soup_tmp = BeautifulSoup(html, 'html.parser')
    tmp = soup_tmp.select_one('.addy').get_text().split(',')
    price = tmp[0].split()[0][:-1]
    price_list.append(price)
    addr = ' '.join(tmp[0].split()[1:])
    
    if len(tmp) == 1:
        addr_list.append(addr)
        tel_list.append(' ')
        hp_list.append(' ')
    elif len(tmp) == 2:
        if addr.find('Multiple') >= 0:
            addr_list.append(' ')
            tel_list.append(' ')
            hp_list.append(tmp[1])
        else :
            addr_list.append(addr)
            tel_list.append(tmp[1].strip())
            hp_list.append(' ')
    elif len(tmp) == 3:
        addr_list.append(addr)
        tel_list.append(tmp[1].strip())
        hp_list.append(tmp[2])
    elif len(tmp) == 4:
        addr += ', ' + tmp[1]
        addr_list.append(addr)
        tel_list.append(tmp[2].strip())
        hp_list.append(tmp[3])

In [125]:
df2 = pd.DataFrame({
    'Rank':rank_list,
    'Cafe':cafe_list,
    'Menu':menu_list,
    'Price':price_list,
    'Address':addr_list,
    'Tel':tel_list,
    'HomePage':hp_list
})
df2.head()

,Rank,Cafe,Menu,Price,Address,Tel,HomePage
0,1,Old Oak Tap,BLT,$10,2109 W. Chicago Ave.,773-772-0406,theoldoaktap.com
1,2,Au Cheval,Fried Bologna,$9,800 W. Randolph St.,312-929-4580,aucheval.tumblr.com
2,3,Xoco,Woodland Mushroom,$9.50,445 N. Clark St.,312-334-3688,rickbayless.com
3,4,Al’s Deli,Roast Beef,$9.40,"914 Noyes St., Evanston",847-475-9400,alsdeli.net
4,5,Publican Quality Meats,PB&L,$10,825 W. Fulton Mkt.,312-445-8977,publicanqualitymeats.com


In [85]:
df2.to_csv('Chicago2.csv', sep=',', encoding='utf8')

### 위치를 지도에 표시하기

In [112]:
import numpy as np
import folium
import googlemaps

In [142]:
key_fd = open('googlemapskey.txt', mode='r')
gmaps_key = key_fd.read(100)
key_fd.close()

In [143]:
gmaps = googlemaps.Client(key=gmaps_key)

In [147]:
lat = []
lng = []

for n in tqdm_notebook(df2.index):
    if df2['Address'][n].find(' ') != 0:
        target_name = df2['Address'][n]+', '+'Chicago'
        gmaps_output = gmaps.geocode(target_name)
        location_output = gmaps_output[0].get('geometry')
        lat.append(location_output['location']['lat'])
        lng.append(location_output['location']['lng'])
    else:
        lat.append(np.nan)
        lng.append(np.nan)

ApiError: REQUEST_DENIED (This API project is not authorized to use this API.)

In [124]:
lat

[]

In [ ]:
df2['Lattitude'] = lat
df2['Longitude'] = lng
df2.head()

In [ ]:
# mapping => 지도를 중심으로 보이게 해준다.
# Marker => 지도의 마커
mapping = folium.Map(location=[df2['Lattitude'].mean(),
                                df2['Longitude'].mean()],zoom_start=11)
folium.Marker([df2['Lattitude'].mean(), df2['Longitude'].mean()],
                                        popup='center').add_to(mapping)
mapping

In [ ]:
mapping = folium.Map(location=[df2['Lattitude'].mean(), df2['Longitude'].mean()],
    zoom_start=11)

for n in df2.index:
    if df2['Address'][n].find(' ') != 0:
        folium.Marker([d2f['Lattitude'].mean(), df2['Longitude'].mean()],
            popup=df2['Cafe'][n]).add_to(mapping)